In [ ]:
import numpy as np
import os, sys
import math
from scipy import ndimage, misc
from astropy.coordinates import Angle
from astropy.io import fits
import matplotlib.pyplot as plt
from scipy import linalg
import csv
import pickle
from matplotlib.colors import LogNorm
from matplotlib import colors
import configparser
import json
from dotenv import load_dotenv, find_dotenv
import pandas as pd
import logging
%matplotlib inline
power = 3
power_dir = "output_neid_order_trace"
# power_dir = "output_paras_order_trace"

In [ ]:
from modules.order_trace.src.alg import OrderTraceAlg
TEST_DIR = '/Users/cwang/documents/KPF/KPF-Pipeline/AlgorithmDev_07122020/test_data_04032020/'
load_dotenv()
KPF_TESTDATA = os.getenv('KPFPIPE_TEST_DATA') + '/'
print(KPF_TESTDATA)
MODULE_DIR = '../../modules/order_trace/'

In [ ]:
from tests.regression import test_order_trace
# add __init__.py tentatively for testing the following functions.
test_order_trace.test_init_exceptions()

In [ ]:

test_order_trace.test_locate_clusters_paras()
test_order_trace.test_form_clusters_paras()
test_order_trace.test_advanced_cluster_cleaning_paras()
test_order_trace.test_clean_clusters_on_borders_paras()
test_order_trace.test_merge_clusters_and_clean_paras()
test_order_trace.test_find_widths_paras()


In [ ]:
# test_order_trace.test_extract_order_trace_paras()

In [ ]:
# input: spectral fits is from dropbox KPF-Pipeline-TestData/order_trace_test
spectral_fits=  KPF_TESTDATA + 'polygon_clipping_test/paras_data/paras.flatA.fits'
output_dir = MODULE_DIR + 'results/result_data'
output_img_dir = MODULE_DIR + 'results/result_img'

# output
cluster_info_xy_csv = output_dir +'/cluster_xy_info_paras.csv'

cluster_clean_fits = output_img_dir+'/cluster_form_paras.fits'
cluster_info_clean_csv = output_dir + '/cluster_form_info_paras.csv'

cluster_after_removal_fits = output_img_dir + '/cluster_clean_paras.fits'
cluster_info_after_removal_csv = output_dir + '/cluster_clean_info_paras.csv'

cluster_border_fits = output_img_dir +'/cluster_clean_border_paras.fits'
cluster_info_border_csv = output_dir + '/cluster_clean_border_info_paras.csv'

cluster_merge_fitting = output_img_dir+'/cluster_merge_paras.fits'
cluster_info_merge_fitting_csv = output_dir+'/cluster_merge_info_paras.csv'

result_csv = output_dir + '/cluster_curve_paras.csv'
result_poly_width_csv =  TEST_DIR + 'order_trace_test/'+power_dir+'/output/paras_result_poly_3sigma_gaussian_pixel_'+str(power)+'.csv'

In [ ]:
# input: spectral fits is from dropbox KPF-Pipeline-TestData/NEIData/FLAT
spectral_fits= KPF_TESTDATA + '/NEIDdata/FLAT/stacked_2fiber_flat.fits'
output_dir = MODULE_DIR + 'results/result_data'
output_img_dir = MODULE_DIR + 'results/result_img'

# output
cluster_info_xy_csv = output_dir +'/cluster_xy_info_neid.csv'

cluster_clean_fits = output_img_dir+'/cluster_form_neid.fits'
cluster_info_clean_csv = output_dir + '/cluster_form_info_neid.csv'

cluster_after_removal_fits = output_img_dir + '/cluster_clean_neid.fits'
cluster_info_after_removal_csv = output_dir + '/cluster_clean_info_neid.csv'

cluster_border_fits = output_img_dir +'/cluster_clean_border_neid.fits'
cluster_info_border_fits = output_dir + '/cluster_clean_border_info_neid.csv'

cluster_merge_fitting = output_img_dir+'/cluster_merge_neid.fits'
cluster_info_merge_fitting_csv = output_dir+'/cluster_merge_info_neid.csv'

result_csv = output_dir + '/cluster_curve_neid.csv'
result_poly_width_csv =  TEST_DIR + 'order_trace_test/'+power_dir+'/output/neid_result_poly_3sigma_gaussian_pixel_'+str(power)+'.csv'
#result_poly_width_csv =  output_dir + 'order_trace_test/'+ power_dir+'/output/neid_result_poly_3sigma_gaussian_pixel_'+str(power)+'.csv'

In [ ]:
# copy of start_logger from logger.py
def get_level(lvl:str) -> int:
    if lvl == 'debug': return logging.DEBUG
    elif lvl == 'info': return logging.INFO
    elif lvl == 'warning': return logging.WARNING
    elif lvl == 'error': return logging.ERROR
    elif lvl == 'critical': return logging.CRITICAL
    else: return logging.NOTSET

def start_logger(logger_name: str, config: str):
    if config is None: 
        # a config file is not provided, so don't start logger
        print('[{}] missing log configuration...not starting a new logger'.format(
            logger_name))
        return None
    config_obj = configparser.ConfigParser()
    res = config_obj.read(config)
    if res == []:
        return None

    log_cfg = config_obj['LOGGER']

    log_start = log_cfg.get('start_log', False)
    log_path = log_cfg.get('log_path', 'log')
    log_lvl = log_cfg.get('log_level', logging.WARNING)
    log_verbose = log_cfg.getboolean('log_verbose', True)
    # logger.setLevel(get_level(log_lvl))
        
    # if log_start:
    #     # setup a log format
    #     formatter = logging.Formatter('[%(name)s][%(levelname)s]:%(message)s')
    #     # setup a log file
    #     f_handle = logging.FileHandler(log_path, mode='w') # logging to file
    #     f_handle.setLevel(get_level(log_lvl))
    #     f_handle.setFormatter(formatter)
    #     logger.addHandler(f_handle)

    #     if log_verbose: 
    #         # also print to terminal 
    #         s_handle = logging.StreamHandler()
    #         s_handle.setLevel(get_level(log_lvl))
    #         s_handle.setFormatter(formatter)
    #         logger.addHandler(s_handle)
    # return logger


    logger = logging.getLogger(logger_name)
    logger.setLevel(get_level(log_lvl))
    logger.propagate = False

    formatter = logging.Formatter('[%(name)s][%(levelname)s]:%(message)s')
    s_handle = logging.StreamHandler()
    s_handle.setLevel(get_level(log_lvl))
    s_handle.setFormatter(formatter)
    logger.addHandler(s_handle)
    return logger


In [ ]:
def plot_imshow(img):
    if (np.amax(img) == 1) and (np.amin(img) == 0):
        print('is bw image')
        im = plt.imshow(img * -1, cmap='gray')
    else:    
        im = plt.imshow(img, cmap='gray', norm=LogNorm())
    return im    

In [ ]:
# plot image from fits by setting area xmin, xmax, ymin, ymax
def plot_img(img, ymin, ymax, p_w=20, p_h=20, xmin=None, xmax=None, title="", aspect=None):
    #if is_bw is True:
    #    img = convert_to_bw(img)
    plt.figure(figsize=(p_w, p_h), frameon=False)
    plt.subplot(1, 1, 1)
    if xmin is None:
        xmin = 0
    if xmax is None:
        h, w = np.shape(img)
        xmax = w-1
    s_img = img[:, :]
    im = plot_imshow(s_img)

    #im = plt.imshow(s_img, cmap='gray')
    plt.ylim(ymin, ymax)
    plt.xlim(xmin, xmax)
    plt.title(title)
    if aspect is not None:
        plt.axes().set_aspect(aspect)
    plt.show()

In [ ]:
def make_fits(data, output_fits):
    hdu = fits.PrimaryHDU(data)
    hdu.writeto(output_fits, overwrite=True)

In [ ]:
# make image data in 2D based on selected clusters
def make_2D_data(index, x, y, nx, ny, selected_clusters=None):
    imm = np.zeros((ny, nx), dtype=np.uint8)
    
    if selected_clusters is None:
        selected_clusters = np.arange(1, np.amax(index)+1, dtype=int)

    for idx in selected_clusters:
        crt_idx = np.where(index == idx)[0]
        imm[y[crt_idx], x[crt_idx]] = 1
        
    return imm

In [ ]:
# make fits on 2D of all clusters
def make_cluster_fits(index, x, y, nx, ny, fits_path=None):
    imm = make_2D_data(index, x, y,  nx, ny)
    if fits_path is not None:
        make_fits(imm, fits_path)
        ind_max = np.amax(index)
        print('there are '+str(ind_max)+' clusters in total in fits, '+fits_path)
    return imm

In [ ]:
# make fits on cluster info (index, x, y)
def make_cluster_info_fits(file_path, nx, ny, x_idx, y_idx, index=None):
    size = x_idx.size
    cluster_data = np.zeros((2, size), dtype=int) if index is None else np.zeros((3, size), dtype=int)
    cluster_data[0, :] = x_idx
    cluster_data[1, :] = y_idx
    if index is not None:
        cluster_data[2, :] = index
    hdu = fits.PrimaryHDU(cluster_data)
    hdu.header['indexmax'] = np.amax(index) if index is not None else 0
    hdu.header['width'] = nx
    hdu.header['height'] = ny
    hdu.writeto(file_path, overwrite=True)
    
def get_cluster_info_fits(file_path: str):
    cluster_info, cluster_info_head = fits.getdata(file_path, header=True)
    x = cluster_info[0, :].astype(int)
    y = cluster_info[1, :].astype(int)
    index_list = None
    if cluster_info_head['indexmax'] != 0:
        index_list = cluster_info[2, :].astype(int)

    return x, y, index_list
    

In [ ]:
# make fits on cluster info (index, x, y)
def make_cluster_info_csv(file_path, nx, ny, x_idx, y_idx, index=None):
    size = x_idx.size
    cluster_data = np.zeros((size, 2), dtype=int) if index is None else np.zeros((size, 3), dtype=int)
    cluster_data[:, 0] = x_idx
    cluster_data[:, 1] = y_idx
    if index is not None:
        cluster_data[:, 2] = index
    df = pd.DataFrame(cluster_data)
    df.to_csv(file_path, header=False, index=False)     
    
def get_cluster_info_csv(file_path: str):
    x = None
    y = None
    index = None
    if os.path.isfile(file_path):
        df = pd.read_csv(file_path, header=None, index_col=None)
        row, col = np.shape(df.values)
        x = df.values[:, 0].astype(int) if col >= 1 else None
        y = df.values[:, 1].astype(int) if col >= 2 else None
        index = df.values[:, 2].astype(int) if col >= 3 else None
    return x, y, index

In [ ]:
# plot polynomial fitting curve on top of given 2D image
# the cluster orders is settable by order_set
def plot_poly_trace(imm, total_order, coeffs_orders, max_x, max_y, size=20, order_set=None, \
                    title=None, background=False, widths=None, aspect=None, xmin=None, xmax=None, ymin=None, ymax=None):
    plt.figure(figsize=(size,size))
    plt.subplot(1, 1, 1)
    im = plot_imshow(imm)
    #im = plt.imshow(imm, cmap='gray', norm=LogNorm())
    
    if order_set is None:
        orders = list(range(1, total_order+1))
    else:
        orders = order_set
        
    x_dist = max_x//20    
                   
    for o_idx, order in enumerate(orders):
        if (background is not False):
            x_val = np.arange(0, max_x)
            # y value on x range
            y_val = np.polyval(coeffs_orders[order, 0:power+1], x_val)
            plt.plot(x_val, y_val, 'b--')
        #print("x range: ", coeffs_orders[order, power+1], coeffs_orders[order, power+2])    
        # x range
        x_val = np.arange(coeffs_orders[order, power+1], coeffs_orders[order, power+2]+1)
        # y value on x range
        y_val = np.polyval(coeffs_orders[order, 0:power+1], x_val)
        plt.plot(x_val, y_val, 'r--')
        
        if widths is not None:
            y_val_bottom = y_val-widths[o_idx][0]
            plt.plot(x_val, y_val_bottom, 'g--')
            y_val_top = y_val+widths[o_idx][1]
            plt.plot(x_val, y_val_top, 'g--')
        
        # show number of cluster
        s = ((order%15)+1)*x_dist
        if s >= x_val.size:
            dem = int((coeffs_orders[order, power+2] - coeffs_orders[order, power+1])//5)
            s = dem*((order%4)+1)
            #s = x_val.size//2
        plt.text(x_val[s], y_val[s], str(order), fontsize=12, color='b', fontweight='bold', horizontalalignment='center', verticalalignment='center')
        
    if title is not None:
        plt.title(title, fontsize=12)
    x1 = 0 if xmin is None else xmin
    x2 = max_x if xmax is None else xmax
    y1 = 0 if ymin is None else ymin
    y2 = max_y if ymax is None else ymax
    
    plt.ylim(y1, y2)
    plt.xlim(x1, x2)
    if aspect is not None:
        plt.axes().set_aspect(aspect)
    
    plt.show()
    #plt.colorbar(im, fraction=0.046, pad=0.04)

In [ ]:
def to_str(afloat):
    new_str = f"{afloat:.4f}"
    return new_str

In [ ]:
# json save and load
def save_obj(obj, filename):
    with open(filename, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

## Usage: Using OrderTarceAlg to extract from the given spectral fits (for NEID or PARAS)

In [ ]:
fits_header = fits.open(spectral_fits)
import pdb;pdb.set_trace()
config_file = MODULE_DIR + 'configs/default.cfg'
logger = start_logger("OrderTraceAlg", config_file)

width_p = 3
config = configparser.ConfigParser()
config.read(config_file)

width_app = '_width_'+str(width_p) if width_p > 0 else '' 

order_t = OrderTraceAlg(fits_header[0].data, config, logger)

if 'neid' in power_dir:
    csv_file = 'neid_poly_3sigma_gaussian_pixel_' + str(power)+ width_app +'.csv'
else:
    csv_file = 'paras_poly_3sigma_gaussian_pixel_'+ str(power)+ width_app +'.csv'
    
imm_spec, nx, ny = order_t.get_spectral_data()
cluster_info = order_t.extract_order_trace(power_for_width_estimation = width_p, show_time=True, print_debug='')
output_df_to_csv  = result_csv;
df = cluster_info['order_trace_result']
df.to_csv(index=False, path_or_buf=output_df_to_csv, header=False)
#order_t.write_cluster_info_to_csv(cluster_info['widths'], cluster_info['coeffs'], result_poly_width_csv)

In [ ]:
# plot the order result and polynomial fits on top of the identified order
new_x = cluster_info['cluster_x']
new_y = cluster_info['cluster_y']
new_index = cluster_info['cluster_index']
_, nx, ny = order_t.get_spectral_data()

cluster_imm = order_t.make_2d_data(new_index, new_x, new_y)
new_coeffs, errors = order_t.curve_fitting_on_all_clusters(new_index, new_x, new_y)
max_index = np.amax(new_index)

print(np.shape(new_coeffs))
print(max_index)
plot_poly_trace(cluster_imm, max_index, new_coeffs, nx, ny)

## Extracting order trace step by step
### excute the cells from step 1 to step 7 and get visual output for each step

## 1. load spectral file

In [ ]:
fits_header = fits.open(spectral_fits)
config_file = MODULE_DIR + 'configs/default.cfg'
config = configparser.ConfigParser()
config.read(config_file)
flat_data = fits_header[0].data
total_row, total_col = np.shape(flat_data)
order_t = OrderTraceAlg(flat_data, config)
#order_t = OrderTraceAlg(flat_data[20:30, 20:30])
#order_t = OrderTraceAlg(flat_data, config )
imm_spec, nx, ny = order_t.get_spectral_data()
spe_info = {'data': imm_spec, 'nx': nx, 'ny': ny}

order_t.add_file_logger("") 
print('row: ', spe_info['ny'], ' column: ', spe_info['nx'])
plot_img(imm_spec, 0, ny-1)
power = order_t.get_poly_degree()
print('power: ', power)

## 2. locate_clusters() -- find cluster pixels  and make fits

In [ ]:
cluster_xy = order_t.locate_clusters()
yy = np.shape(cluster_xy['cluster_image'])[0]
plot_img(cluster_xy['cluster_image'], 0, yy-1)

In [ ]:
make_cluster_info_csv(cluster_info_xy_csv, nx, ny, cluster_xy['x'], cluster_xy['y'])

## 3. form_clusters() or collect_clusters() + remove_cluster_by_size() + reorganize_index()

In [ ]:
#cluster_info, dict
#cluster_info = order_t.collect_clusters(list(), list())
cluster_info = order_t.collect_clusters(cluster_xy['x'], cluster_xy['y'])

In [ ]:
# assign index value to cluster_info['index'], where cluster_info['index'] is the same size as cluster_xy['x']
cluster_info = order_t.remove_cluster_by_size(cluster_info, cluster_xy['x'], cluster_xy['y'])

In [ ]:
# remove unassigned index
x, y, index_r = order_t.reorganize_index(cluster_info['index'], cluster_xy['x'], cluster_xy['y'])
imm = order_t.make_2d_data(index_r, x, y)   # show image  and make fits and info fits 
plot_img(imm, 0, np.shape(imm)[0]-1)

### save data from the result of 3 (optional)

In [ ]:
imm=make_cluster_fits(index_r, x, y, nx, ny, cluster_clean_fits)
make_cluster_info_csv(cluster_info_clean_csv, nx, ny, x, y, index_r)
plot_img(imm, 0, np.shape(imm)[0]-1)

### reload clean fits and info fits of step 3. (optional)

In [ ]:
# (optional) reload saved fits 
imm, hdr = fits.getdata(cluster_clean_fits, header=True)
ny, nx = np.shape(imm)
x, y, index_r = get_cluster_info_csv(cluster_info_clean_csv)
plot_img(imm, 0, np.shape(imm)[0]-1)

## 4. advanced_cluster_cleaning_handler() -- cluster cleaning to remove noisy clusters

In [ ]:
new_x, new_y, new_index, all_status = order_t.advanced_cluster_cleaning_handler(index_r, x, y)

imm = order_t.make_2d_data(new_index, new_x, new_y)   # show image  and make fits and info fits 
plot_img(imm, 0, np.shape(imm)[0]-1)

### save advanced cleaning result of step 4 to fits and info fits (optional)

In [ ]:
plot_img(make_cluster_fits(new_index, new_x, new_y, nx, ny), 0, ny-1)

In [ ]:
imm_advanced = make_cluster_fits(new_index, new_x, new_y, nx, ny, cluster_after_removal_fits)
make_cluster_info_csv(cluster_info_after_removal_csv, nx, ny, new_x, new_y, new_index)

plot_img(imm_advanced, 0, np.shape(imm_advanced)[0]-1)

### reload advanced cleaning results from result of step 4 (optional)

In [ ]:
new_imm_after_removal, hdr = fits.getdata(cluster_after_removal_fits, header=True)
plot_img(new_imm_after_removal, 0, np.shape(new_imm_after_removal)[0]-1)
new_x, new_y, new_index = get_cluster_info_csv(cluster_info_after_removal_csv)

## 5. clean_clusters_on_borders() or clean_clusters_on_border() for top and bottom border 

In [ ]:
x, y, index_b = order_t.clean_clusters_on_border(new_x, new_y, new_index, 0)
print(len(new_index))
print(len(index_b))

new_x, new_y, new_index = order_t.clean_clusters_on_border(x, y, index_b, ny-1)
print(len(index_b))
print(len(new_index))
imm = order_t.make_2d_data(new_index, new_x, new_y)   # show image  and make fits and info fits 
plot_img(imm, 0, np.shape(imm)[0]-1)

###  saving result of step 5 to fits and info fits (optional)

In [ ]:
imm_border = make_cluster_fits(new_index, new_x, new_y, nx, ny, cluster_border_fits)
make_cluster_info_csv(cluster_info_border_csv, nx, ny, new_x, new_y, new_index)
plot_img(imm_border, 0, ny-1)

### reload result of step 5 fits and info fits (optional)

In [ ]:
imm_border, hdr = fits.getdata(cluster_border_fits, header=True)
plot_img(imm_border, 0, np.shape(imm_border)[0]-1)

new_x, new_y, new_index = get_cluster_info_csv(cluster_info_border_csv)

## plot order trace of step 5

In [ ]:
# prepare for step 6
imm_border = make_2D_data(new_index, new_x, new_y, nx, ny)
new_coeffs, errors = order_t.curve_fitting_on_all_clusters(new_index, new_x, new_y)
max_index = np.amax(new_index)

print(np.shape(new_coeffs))
print(max_index)
plot_poly_trace(imm_border, max_index, new_coeffs, nx, ny)

## 6. merge_cluster() -  Merging clusters

In [ ]:
new_x, new_y, new_index = order_t.merge_clusters_and_clean(new_index, new_x, new_y)
imm = order_t.make_2d_data(new_index, new_x, new_y)   # show image  and make fits and info fits 
plot_img(imm, 0, np.shape(imm)[0]-1)

### save merge results of step 6 to fits and info fits (optional)

In [ ]:
# store after merge fits and fits info
imm = make_cluster_fits(new_index, new_x, new_y, nx, ny, cluster_merge_fitting)
make_cluster_info_csv(cluster_info_merge_fitting_csv, nx, ny, new_x, new_y, new_index)
plot_img(imm, 0, np.shape(imm)[0]-1)

### reload merge result of step 6  (optional)

In [ ]:
imm, hdr = fits.getdata(cluster_merge_fitting, header=True)
plot_img(imm, 0, np.shape(imm)[0]-1)

new_x, new_y, new_index = get_cluster_info_csv(cluster_info_merge_fitting_csv)

### plot order trace of result of step 6

In [ ]:
new_coeffs, errors = order_t.curve_fitting_on_all_clusters(new_index, new_x, new_y)
max_index = np.amax(new_index)    
plot_poly_trace(imm, max_index, new_coeffs, nx, ny)

## 7. find_all_cluster_widths() -- find top and bottom widths of the orders

In [ ]:
cluster_widths, cluster_coeffs =  order_t.find_all_cluster_widths(new_index, new_x, new_y, power_for_width_estimation=3)

### output to Pandas dataframe and then to csv  

In [ ]:
cluster_coeffs[:, 0:4]

In [ ]:
result_poly_width_csv = result_csv
order_t.write_cluster_info_to_csv(cluster_widths, cluster_coeffs, result_poly_width_csv)